## Python Imports

In [ ]:
!pip install accelerate
!pip install transformers
!pip install sentence-transformers
!pip install datasets
!pip install bert_score
!pip install evaluate
!pip install nltk
!pip install bert_score

import accelerate
import torch
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt

import os
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup, TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, AutoModelForSeq2SeqLM, GenerationConfig
from datasets import Dataset, DatasetDict
from accelerate import Accelerator
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import nltk
nltk.download('punkt_tab')
nltk.download('punkt')   # Ensure 'punkt' is downloaded
from nltk.tokenize import word_tokenize
from evaluate import load

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_csv('/content/tqsk3.csv')
tdf = df.iloc[:77]
vdf = df.iloc[77:]

In [ ]:
# print(tedf.Prompt.dtype,
# tedf.Content.dtype,
# tedf['Narrative 1'].dtype)
# tedf['Narrative 1']

In [ ]:
# from transformers import BartTokenizer, BartForConditionalGeneration

# checkpoint = "facebook/bart-large-cnn"
# model = BartForConditionalGeneration.from_pretrained(checkpoint)
# tokenizer = BartTokenizer.from_pretrained(checkpoint)

# model

checkpoint = "google/flan-t5-small"  # Replace 'large' with 'base', 'small', or 'xl' depending on needs
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
def tokenize_function(example):
    inputs = tokenizer(example['Prompt'], max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(example['Explanation'], max_length=80, truncation=True, padding="max_length", return_tensors="pt")

    inputs['labels'] = targets['input_ids']
    return inputs

# df['Prompt'] = np.where(df['Narrative 2'] != 'none',
#                         'Based on the narrative following narrative: ' + df['Narrative 2']+ '  find the summary of the: ' + df['Content'],
#                         'Based on the narrative following narrative: ' + df['Narrative 1']+ '  find the summary of the: ' + df['Content'])

df_train = tdf[['Prompt', 'Explanation']].copy()
df_val = vdf[['Prompt', 'Explanation']].copy()

traindataset = Dataset.from_pandas(df_train)
train_dataset = traindataset.map(tokenize_function, batched=True)

validationdataset = Dataset.from_pandas(df_val)
validation_dataset = validationdataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/77 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [ ]:
# Load the metrics using evaluate
bertscore = load("bertscore")

def compute_metrics(pred):
    pred_logits = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred_str = [tokenizer.decode(ids, skip_special_tokens=True) for ids in pred_ids]

    # Remove ignored index (-100) from labels and replace with pad token id
    labels_ids = np.where(pred.label_ids != -100, pred.label_ids, tokenizer.pad_token_id)
    label_str = [tokenizer.decode(ids, skip_special_tokens=True) for ids in labels_ids]

    # Compute ROUGE score
    bertscore_output = bertscore.compute(predictions=pred_str, references=label_str, lang="en")
    # Return ROUGE and BLEU scores
    return {
        'bertscore_precision': sum(bertscore_output['precision']) / len(bertscore_output['precision']),
        'bertscore_recall': sum(bertscore_output['recall']) / len(bertscore_output['recall']),
        'bertscore_f1': sum(bertscore_output['f1']) / len(bertscore_output['f1']),
    }


# Initialize WandB with disabled mode
import wandb
wandb.init(mode="disabled")


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',           # Directory to save checkpoints
    num_train_epochs=7,               # Number of training epochs
    per_device_train_batch_size=4,
    warmup_steps=10,                  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # Strength of weight decay
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=10,                 # Log every X updates
    save_strategy="steps",            # Save the model at the end of each epoch
    eval_strategy="steps",            # Evaluate at the end of each epoch
    load_best_model_at_end=True,      # Load the best model found during evaluation
    learning_rate=2e-5,               # Learning rate
    save_total_limit=5,
    metric_for_best_model="bertscore_f1",   # Use ROUGE-L to determine the best model
    greater_is_better=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
# 72ec5df7b7871e699a9c5e61b91a0cb74038b3c1

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=140, training_loss=12.913938249860491, metrics={'train_runtime': 88.8401, 'train_samples_per_second': 6.067, 'train_steps_per_second': 1.576, 'total_flos': 200389964070912.0, 'train_loss': 12.913938249860491, 'epoch': 7.0})

In [ ]:
model.save_pretrained("/content/")
tokenizer.save_pretrained("/content/")

('/content/tokenizer_config.json',
 '/content/special_tokens_map.json',
 '/content/spiece.model',
 '/content/added_tokens.json',
 '/content/tokenizer.json')

In [ ]:

# generation_config = GenerationConfig(
#     max_length=150,
#     min_length=56,
#     # early_stopping=True,
#     early_stopping=False,
#     num_beams=4,
#     length_penalty=2.0,
#     no_repeat_ngram_size=3,
#     forced_bos_token_id=0,
#     forced_eos_token_id=2
# )

# Load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/")
tokenizer = AutoTokenizer.from_pretrained("/content/")
generation_config = GenerationConfig.from_pretrained("/content/")

In [ ]:
# tedf

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)


# def summarize_text(text):

#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
#     summary_ids = model.generate(
#         inputs['input_ids'],
#         generation_config=generation_config
#     )
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary

# # tedf['summary'] = tedf['Content'].apply(summarize_text)
# # print(tedf['Content'])
# # tedf['summary'] = tedf['Prompt'].apply(summarize_text)

In [ ]:
# predictions = tedf['summary'].tolist()
# references = tedf['Explanation'].tolist()
# # results = bertscore.compute(predictions=predictions, references=references, lang="en")
# results = bertscore.compute(predictions=predictions, references=references, lang="en")

In [ ]:
# results

In [ ]:
# predictions

In [ ]:
# references

# THIS IS ADDED LATER AFTER THE CODE WORKED SO PLS DONT CHANGE ABOVE THIS

In [ ]:
import requests

# Raw URL for the annotations file
annotations_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-3-dominant-narratives.txt"

# Load the annotations file
df1 = pd.read_csv(annotations_url, sep="\t", header=None, names=['article_id', 'Narrative 1', 'Narrative 2'])

# Base URL for the raw documents
base_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-3-documents/"

# Function to fetch the article content from the raw documents folder
def fetch_file_content(article_id):
    file_url = f"{base_url}{article_id}"
    # print(f"Fetching URL: {file_url}")  # Debugging URL
    response = requests.get(file_url)
    # print(f"Status Code: {response.status_code}")  # Debugging response status
    if response.status_code == 200:
        return response.text
    else:
        return f"File {article_id} not found"


# Fetch article content for each article_id
df1['Content'] = df1['article_id'].apply(fetch_file_content)



# Generate the Prompt column
df1['Prompt'] = np.where(
    df1['Narrative 2'] != 'none',
    'Based on the the: ' + df1['Narrative 2'] + ' Give the summary of the: ' + df1['Content'],
    'Based on the the: ' + df1['Narrative 1'] + ' Give the summary of the: ' + df1['Content'],
)

ledf = df1.copy()

In [ ]:
ledf['Prompt'] = ledf['Prompt'].apply(lambda x: x[:4290] if isinstance(x, str) else x)
print(ledf['Prompt'].apply(len).max())
ledf

4290


article_id                                        Narrative 1  \
0   EN_UA_DEV_100012.txt              URW: Discrediting the West, Diplomacy   
1       EN_CC_200040.txt                  CC: Criticism of climate movement   
2       EN_CC_200070.txt      CC: Criticism of institutions and authorities   
3   EN_UA_DEV_100034.txt                         URW: Overpraising the West   
4       EN_CC_200049.txt       CC: Questioning the measurements and science   
5   EN_UA_DEV_100033.txt                      URW: Speculating war outcomes   
6       EN_CC_200036.txt                  CC: Criticism of climate movement   
7       EN_CC_200079.txt      CC: Criticism of institutions and authorities   
8   EN_UA_DEV_100029.txt                      URW: Speculating war outcomes   
9       EN_UA_DEV_20.txt                          URW: Discrediting Ukraine   
10  EN_UA_DEV_100005.txt              URW: Discrediting the West, Diplomacy   
11      EN_CC_200033.txt                  CC: Criticism of climate movement   
12      EN_CC_200071.txt                  CC: Criticism of climate policies   
13      EN_CC_200046.txt                   CC: Climate change is beneficial   
14  EN_UA_DEV_100026.txt            URW: Negative Consequences for the West   
15     EN_UA_DEV_215.txt                          URW: Discrediting Ukraine   
16      EN_CC_200054.txt  CC: Hidden plots by secret schemes of powerful...   
17  EN_UA_DEV_100013.txt                          URW: Discrediting Ukraine   
18      EN_CC_200077.txt                  CC: Criticism of climate policies   
19      EN_CC_200034.txt                  CC: Criticism of climate movement   
20     EN_UA_DEV_213.txt  URW: Blaming the war on others rather than the...   
21      EN_CC_200047.txt  CC: Hidden plots by secret schemes of powerful...   
22      EN_CC_200081.txt           CC: Controversy about green technologies   
23     EN_UA_DEV_214.txt              URW: Discrediting the West, Diplomacy   
24      EN_CC_200064.txt                  CC: Criticism of climate movement   
25      EN_UA_DEV_22.txt              URW: Discrediting the West, Diplomacy   
26      EN_CC_200035.txt      CC: Criticism of institutions and authorities   
27      EN_CC_200069.txt                  CC: Criticism of climate policies   
28      EN_CC_200065.txt                  CC: Criticism of climate movement   
29  EN_UA_DEV_100002.txt                      URW: Speculating war outcomes   

                                          Narrative 2  \
0   URW: Discrediting the West, Diplomacy: The Wes...   
1                                                none   
2   CC: Criticism of institutions and authorities:...   
3   URW: Overpraising the West: The West belongs i...   
4                                                none   
5   URW: Speculating war outcomes: Russian army is...   
6   CC: Criticism of climate movement: Ad hominem ...   
7   CC: Criticism of institutions and authorities:...   
8   URW: Speculating war outcomes: Russian army is...   
9   URW: Discrediting Ukraine: Situation in Ukrain...   
10                                               none   
11                                               none   
12  CC: Criticism of climate policies: Climate pol...   
13  CC: Climate change is beneficial: CO2 is benef...   
14  URW: Negative Consequences for the West: Sanct...   
15  URW: Discrediting Ukraine: Discrediting Ukrain...   
16  CC: Hidden plots by secret schemes of powerful...   
17  URW: Discrediting Ukraine: Discrediting Ukrain...   
18  CC: Criticism of climate policies: Climate pol...   
19  CC: Criticism of climate movement: Climate mov...   
20  URW: Blaming the war on others rather than the...   
21  CC: Hidden plots by secret schemes of powerful...   
22  CC: Controversy about green technologies: Rene...   
23  URW: Discrediting the West, Diplomacy: The Wes...   
24                                               none   
25  URW: Discrediting the West, Diplomacy: The EU ...   
26  CC: Criticism of inst

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


def summarize_text(text):

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    summary_ids = model.generate(
        inputs['input_ids'],
        generation_config=generation_config
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

ledf['summary'] = ledf['Prompt'].apply(summarize_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
ledf['summary']
print(ledf['summary'].apply(len).max())
ledf.drop(columns=['Narrative 1','Narrative 2','Content', 'Prompt'], inplace=True)

117


In [ ]:
output_txt_path = "mums.txt"  # Replace with the desired TXT file path
ledf.to_csv(output_txt_path, sep="\t", index=False, header=False)